In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/showjoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import the dataset

In [2]:
imdb_data = pd.read_csv('./IMDB_Dataset.csv')
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### Generate the Training and Testing data by dividing the dataset as 4:1

In [4]:
train_reviews=imdb_data.review[:30000]
train_sentiments=imdb_data.sentiment[:30000]

valid_reviews=imdb_data.review[30000:40000]
valid_sentiments=imdb_data.sentiment[30000:40000]

test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]

train_reviews.shape,train_sentiments.shape,valid_reviews.shape,valid_sentiments.shape, test_reviews.shape,test_sentiments.shape


((30000,), (30000,), (10000,), (10000,), (10000,), (10000,))

### Make sure that the split is balanced in general

In [5]:
train_sentiments.value_counts()

positive    15015
negative    14985
Name: sentiment, dtype: int64

### Preprocessing the dataset

In [6]:
# tokenizer = ToktokTokenizer()
# stopword_list=nltk.corpus.stopwords.words('english')

# #Stemming the text, e.g. am, are, is -> be
# def simple_stemmer(text):
#     ps=nltk.porter.PorterStemmer()
#     text= ' '.join([ps.stem(word) for word in text.split()])
#     return text

# train_reviews = train_reviews.apply(simple_stemmer)
# test_reviews = test_reviews.apply(simple_stemmer)

In [7]:
# set stopwords to english
# stop = set(stopwords.words('english'))
# print(stop)

In [8]:
# #removing the stopwords
# def remove_stopwords(text, is_lower_case=False):
#     tokens = tokenizer.tokenize(text)
#     tokens = [token.strip() for token in tokens]
#     if is_lower_case:
#         filtered_tokens = [token for token in tokens if token not in stopword_list]
#     else:
#         filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
#     filtered_text = ' '.join(filtered_tokens)    
#     return filtered_text

# train_reviews = train_reviews.apply(remove_stopwords)
# test_reviews = test_reviews.apply(remove_stopwords)


如果在max_df中超过百分之50的一个词其中大部分都在positive，positive indicator？

 # Naive bayes model
 https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

## Method that take in the training dataset, then return the positive and negative words-log probability dictionary.


In [6]:
'''
Method that take in the training dataset, then return the positive and negative words log probability.
Input: train_reviews: reviews (sentences) for training
       train_sentiments: sentiments (label) for training
       tfidf: boolean variable indicating whether using bow or tfidf
       alpha: laplance smoothing variable, default to be 1.0
       ngram_range: the scale of ngram model will be used, default = (1,1) unigram
return: negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class
        positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class
        mnb: the trained multinomial naive bayes model, later can be used for testing
        transformed_test_reviews: transformed test reviews that later can be used for testing
        vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
'''

def generate_log_prob(train_reviews, train_sentiments, tfidf=False, alpha=1.0, ngram_range = (1,1)):

    if (tfidf):
        #Tfidf vectorizer
        vec=TfidfVectorizer(use_idf=tfidf, ngram_range=ngram_range)
        #transformed train reviews
        transformed_train_reviews=vec.fit_transform(train_reviews)
        #transformed test reviews
        transformed_test_reviews=vec.transform(test_reviews)
    else:
        vec=CountVectorizer(ngram_range=ngram_range)
        transformed_train_reviews=vec.fit_transform(train_reviews)
        transformed_test_reviews=vec.transform(test_reviews)

    #training the model
    mnb = MultinomialNB(alpha=alpha)

    #fitting the naive bayes for bag of words
    mnb = mnb.fit(transformed_train_reviews, train_sentiments)
    negative_log_prob = mnb.feature_log_prob_[0]
    positive_log_prob = mnb.feature_log_prob_[1]

    # Generate two dict: word:log_prob
    negative_word_log_prob_dict = {}
    positive_word_log_prob_dict = {}
    for word, index in vec.vocabulary_.items():
        negative_word_log_prob_dict[word] = negative_log_prob[index]
        positive_word_log_prob_dict[word] = positive_log_prob[index]
    
    return negative_word_log_prob_dict, positive_word_log_prob_dict, mnb, transformed_train_reviews, transformed_test_reviews, vec

## Method that given input word, manually change the weight of the word in naive bayes model

In [7]:
'''
This method will take in a word:scale dict, then take in the negative and positive word:log_probability dict, manually change the weight of the words in the model and the dict
Input: negative_word_change_scale: this is the word-scale dictionary, which specify in the negative class, how much the weight of the word should be changed, For example
                                   if the value is 0.5, we will say the probability of the word in negative class should multiply 0.5, transformed into addition for log prob
       positive_word_change_scale: this is the word-scale dictionary, which specify in the positive class, how much the weight of the word should be changed, For example
                                   if the value is 2, we will say the probability of the word in negative class should multiply 2, transformed into addition for log prob
       model: the trained naive bayes model, which the feature_log_prob_ attribute will be manually changed based on previous two params
       negative_word_log_prob_dict: dictionary that contains the word:log probability pair for negative class, which some values will be changed
       positive_word_log_prob_dict: dictionary that contains the word:log probability pair for positive class, which some values will be changed
       vec: either the tfidfVectorize build from tfidf model or the CountVectorizer build from Bag of word model.
return: negative_word_change_scale: The modified negative dict
        positive_word_change_scale: The modified positive dict
        model: The modified naive bayes model
'''

def change_weight(negative_word_change_scale, positive_word_change_scale, model, negative_word_log_prob_dict, positive_word_log_prob_dict, vec):
    for word, scale in negative_word_change_scale.items():
        # change the weight of words in negative and positive word:log_prob dict
        negative_word_log_prob_dict[word] *= scale

        # change the weight of words in the model
        index_in_model = vec.vocabulary_[word]
        model.feature_log_prob_[0][index_in_model] *= scale

    for word, scale in positive_word_change_scale.items():
        # change the weight of words in negative and positive word:log_prob dict
        positive_word_log_prob_dict[word] *= scale

        # change the weight of words in the model
        index_in_model = vec.vocabulary_[word]
        model.feature_log_prob_[1][index_in_model] *= scale

    return negative_word_log_prob_dict, positive_word_log_prob_dict, model

In [11]:
np.log(12),np.log(1/12)

(2.4849066497880004, -2.4849066497880004)

In [8]:
valid_sentiments.index

RangeIndex(start=30000, stop=40000, step=1)

In [32]:
reverse_vec = {k:v for v,k in vec.vocabulary_.items()}
reverse_vec[1963]

'addtion'

## Method that given input 2 dict, 寻找每个单词正反作用的count，向反作用的单词添加小weight，减弱其作用

In [10]:
from collections import defaultdict

def classify(positive_dict,negative_dict,count_dict:defaultdict,transformed_valid_reviews,valid_sentiments,vec,wrong_label):
    rows = np.nonzero(transformed_valid_reviews)[0]
    columns = np.nonzero(transformed_valid_reviews)[1]
    indices = zip(rows,columns)
    start_index = valid_sentiments.index.start
    # find word using index
    reverse_vec = {k:v for v,k in vec.vocabulary_.items()}
    #travase - only wrong case
    if wrong_label:
        indices = filter(lambda index:index[0] in wrong_label,indices)
        for row,column in indices:
            label = valid_sentiments[start_index+row]
            word = reverse_vec[column]
            count = transformed_valid_reviews[row,column]
            #compare
            if label == 'positive':
                #起正作用情况
                if positive_dict[word] > negative_dict[word]:
                    count_dict[word]['good'] += count
                #起负作用情况
                elif positive_dict[word] < negative_dict[word]:
                    count_dict[word]['bad'] += count
            elif label == 'negative':
                #起正作用情况
                if positive_dict[word] < negative_dict[word]:
                    count_dict[word]['good'] += count
                #起负作用情况
                elif positive_dict[word] > negative_dict[word]:
                    count_dict[word]['bad'] += count
    else: # all case
        for row,column in indices:
            label = valid_sentiments[start_index+row]
            word = reverse_vec[column]
            count = transformed_valid_reviews[row,column]
            #compare
            if label == 'positive':
                #起正作用情况
                if positive_dict[word] > negative_dict[word]:
                    count_dict[word]['good'] += count
                #起负作用情况
                elif positive_dict[word] < negative_dict[word]:
                    count_dict[word]['bad'] += count
            elif label == 'negative':
                #起正作用情况
                if positive_dict[word] < negative_dict[word]:
                    count_dict[word]['good'] += count
                #起负作用情况
                elif positive_dict[word] > negative_dict[word]:
                    count_dict[word]['bad'] += count        
    return count_dict

In [13]:
def reweight_count_wrong(positive_dict,negative_dict,transformed_valid_reviews,valid_sentiments,vec,wrong_label = []) -> dict:
    count_dict = defaultdict(lambda: defaultdict(int))
    count_dict = classify(positive_dict,negative_dict,count_dict,transformed_valid_reviews,valid_sentiments,vec,wrong_label)
    #initialize reweight_dict
    #reweight_dict = defaultdict(lambda: 1)
    #filter bad > good, hyperparameter - set 怎么算要reweight
    new_count_dict = {i:count_dict[i] for i in filter(lambda k:count_dict[k]['bad'] > count_dict[k]['good'],count_dict.keys())}
    #return count_dict, hyperparameter - set reweight成多少
    #transfer to bad+good/bad-good
    transfer_count_dict = {k:(new_count_dict[k]['bad'] + new_count_dict[k]['good'])/(new_count_dict[k]['bad'] - new_count_dict[k]['good']) for k in new_count_dict.keys()}
    #normalization
    d_range = max(transfer_count_dict.values()) -min(transfer_count_dict.values())
    d_min = min(transfer_count_dict.values())
    test_reweight_dict = {k:(v-d_min)/d_range for k,v in transfer_count_dict.items()}
    return test_reweight_dict,count_dict
                    
                    

### Predict the result using bag of word methods

## method 1 using on whole dataset

In [12]:
neg, pos, mnb, transformed_train_reviews, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, alpha = 0.05)
transformed_valid_reviews=vec.transform(valid_reviews)

In [21]:
#产生调优测试案例

In [14]:
reweight_dict,count_dict = reweight_count_wrong(pos,neg,transformed_valid_reviews,valid_sentiments,vec)

In [15]:
test_reweight_dict = reweight_dict

In [16]:
#test_reweight_dict = {k:0.0001 for k,v in reweight_dict.items()}

In [17]:
'''
#test_reweight_dict
new_count_dict = {i:count_dict[i] for i in filter(lambda k:count_dict[k]['bad'] > count_dict[k]['good'],count_dict.keys())} 
#transfer to bad+good/bad-good
transfer_count_dict = {k:(new_count_dict[k]['bad'] + new_count_dict[k]['good'])/(new_count_dict[k]['bad'] - new_count_dict[k]['good']) for k in new_count_dict.keys()}
#normalization
d_range = max(transfer_count_dict.values()) -min(transfer_count_dict.values())
d_min = min(transfer_count_dict.values())
test_reweight_dict = {k:(v-d_min)/d_range for k,v in transfer_count_dict.items()}
'''

"\n#test_reweight_dict\nnew_count_dict = {i:count_dict[i] for i in filter(lambda k:count_dict[k]['bad'] > count_dict[k]['good'],count_dict.keys())} \n#transfer to bad+good/bad-good\ntransfer_count_dict = {k:(new_count_dict[k]['bad'] + new_count_dict[k]['good'])/(new_count_dict[k]['bad'] - new_count_dict[k]['good']) for k in new_count_dict.keys()}\n#normalization\nd_range = max(transfer_count_dict.values()) -min(transfer_count_dict.values())\nd_min = min(transfer_count_dict.values())\ntest_reweight_dict = {k:(v-d_min)/d_range for k,v in transfer_count_dict.items()}\n"

In [98]:
#transfer_count_dict.values()

In [103]:
#test_reweight_dict

In [18]:
#test_reweight_dict.keys()

In [19]:
len(test_reweight_dict)

12469

In [20]:
count_dict['silver']

defaultdict(int, {'good': 22, 'bad': 34})

This block represent the length of the vocabulary

In [21]:
len(vec.vocabulary_.items())

82453

Following two blocks shows the word:log_prob key value pair for each phrase in the vocabulary

In [28]:
#pos

In [29]:
#neg

Following block use the trained model to do the prediction

In [22]:
mnb_bow_predict = mnb.predict(transformed_test_reviews)


To check the corresponding test reviews log probability and original probability.


NOTICE: this probability was being normalized across all classes, so the log_probability was not the summation of log class probability and log probability of all phrases in the sentence. You can see that the summation of predict_proba() method is 1. The predict_log_proba() method calculate the log probability based on the normalized one.

In [23]:
mnb.predict_proba(transformed_test_reviews[0]), mnb.predict_log_proba(transformed_test_reviews[0])

(array([[1.00000000e+00, 8.17095253e-17]]),
 array([[  0.        , -37.04336109]]))

This predict() method will return the predict value of given transformed review

In [32]:
mnb.predict(transformed_test_reviews[0]), mnb.predict(transformed_test_reviews[1])

(array(['negative'], dtype='<U8'), array(['negative'], dtype='<U8'))

This class_log_prior_ attribute of the MultinomialNB() model gives the prior class probability. In the order of the classes_ attribute

In [33]:
mnb.classes_, mnb.class_log_prior_

(array(['negative', 'positive'], dtype='<U8'),
 array([-0.69279724, -0.69349724]))

Now we put the entire training set into the trained Naive Bayes model, and ge the score

In [24]:
mnb_bow_predict = mnb.predict(transformed_test_reviews)
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)


mnb_bow_score : 0.843


In [25]:
mnb.feature_log_prob_

array([[-10.87879434,  -9.67537681, -14.97393224, ..., -14.97393224,
        -14.97393224, -14.97393224],
       [-11.04874497, -10.00137729, -18.03438678, ..., -18.03438678,
        -18.03438678, -18.03438678]])

In [27]:
#测valid
mnb_bow_predict = mnb.predict(transformed_valid_reviews)
mnb_bow_score = accuracy_score(valid_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)


mnb_bow_score : 0.8431


In [28]:
#change weight to test accuracy change
new_neg, new_pos, new_mnb = change_weight(test_reweight_dict, test_reweight_dict, mnb, neg, pos, vec)
mnb_bow_predict = new_mnb.predict(transformed_test_reviews)
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.847


In [29]:
new_mnb.feature_log_prob_

array([[ -0.09831978,  -9.67537681, -14.97393224, ..., -14.97393224,
        -14.97393224, -14.97393224],
       [ -0.09985575, -10.00137729, -18.03438678, ..., -18.03438678,
        -18.03438678, -18.03438678]])

In [30]:
#测valid
mnb_bow_predict = mnb.predict(transformed_valid_reviews)
mnb_bow_score = accuracy_score(valid_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.8808


Classification report

In [38]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

              precision    recall  f1-score   support

    Positive       0.83      0.88      0.85      4993
    Negative       0.87      0.82      0.84      5007

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [29]:
wrong_labeled_NB_BOW = ""

for i in range(mnb_bow_predict.size):
    start_index = test_sentiments.index.start
    if(mnb_bow_predict[i] != test_sentiments[start_index + i]):
        wrong_labeled_NB_BOW += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("Naive Bayes - Bag of Words Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_NB_BOW)
f.close()

## method 2 try on only wrong labeled set

In [219]:
#another version - only wrong labeled
neg, pos, mnb, transformed_train_reviews, transformed_test_reviews, vec= generate_log_prob(train_reviews, train_sentiments, alpha = 0.05)
mnb_bow_predict = mnb.predict(transformed_test_reviews)
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.8488


In [165]:
mnb_bow_train = mnb.predict(transformed_train_reviews)
mnb_bow_score = accuracy_score(train_sentiments, mnb_bow_train)
print("mnb_bow_score :",mnb_bow_score)

mnb_bow_score : 0.918925


In [168]:
wrong_labeled_train = []
for i in range(mnb_bow_train.size):
    start_index = train_sentiments.index.start
    if(mnb_bow_train[i] != train_sentiments[start_index + i]):
        wrong_labeled_train.append(start_index + i)

In [182]:
len(wrong_labeled_train)

3243

In [31]:
#reweight_dict_wrong,count_dict_wrong = reweight_count_wrong(pos,neg,transformed_train_reviews,train_sentiments,vec,wrong_labeled_train)


In [ ]:
#change weight to test accuracy change
new_neg, new_pos, new_mnb = change_weight(reweight_dict_wrong, reweight_dict_wrong, mnb, neg, pos, vec)
mnb_bow_predict = new_mnb.predict(transformed_test_reviews)
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)

### Predict the model using TF-IDF method

In [214]:
neg, pos, mnb, transformed_train_reviews,transformed_test_reviews, vec = generate_log_prob(train_reviews, train_sentiments, tfidf = True, alpha = 0.05)

In [215]:
mnb_tfidf_predict = mnb.predict(transformed_test_reviews)
mnb_tfidf_score = accuracy_score(test_sentiments, mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)


mnb_tfidf_score : 0.8623


In [207]:
reweight_dict_idf,count_dict_idf = reweight_count_wrong(pos,neg,transformed_train_reviews,train_sentiments,vec)

In [216]:
test_reweight_dict_idf = {k:0.5 for k,v in reweight_dict_idf.items()}

In [217]:
#change weight to test accuracy change
new_neg, new_pos, new_mnb = change_weight(test_reweight_dict_idf, test_reweight_dict_idf, mnb, neg, pos, vec)
mnb_tfidf_predict = new_mnb.predict(transformed_test_reviews)
mnb_tfidf_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_tfidf_score : 0.8491


In [74]:
#Classification report for TF-IDF
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.85      0.87      0.86      4993
    Negative       0.87      0.85      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [75]:
wrong_labeled_NB_TFIDF = ""
for i in range(mnb_tfidf_predict.size):
    start_index = test_sentiments.index.start
    if(mnb_tfidf_predict[i] != test_sentiments[start_index + i]):
        wrong_labeled_NB_TFIDF += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("Naive Bayes - TF-IDF Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_NB_TFIDF)
f.close()

# fastText method

In [76]:
import fasttext
test_reviews.index.stop

50000

In [77]:
#preprocess the data so it can feed into fasttext
def transform_instance(doc, label):
    processed_text = []
    #Prefix the index-ed label with __label__
    for i in range(doc.index.start, doc.index.stop):
        cur_row = "__label__" + label[i] + " " + doc[i]
        processed_text.append(cur_row)
    return processed_text

training_text = transform_instance(train_reviews, train_sentiments)
test_text = transform_instance(test_reviews, test_sentiments)

# Put the training and test dataset into file, so that the fasttext model can read them.
f = open("fasttext.train", "w")
output = ""
for text in training_text:
    output += text
    output += "\n"
f.write(output)
f.close()

f = open("fasttext.test", "w")
output = ""
for text in test_text:
    output += text
    output += "\n"
f.write(output)
f.close()

In [78]:
len(training_text), len(test_text)

(40000, 10000)

In [79]:
model = fasttext.train_supervised(input = "fasttext.train", lr=0.1, epoch=25, wordNgrams=3)

In [80]:
model.save_model("model_movie.bin")

In [81]:
model.test("fasttext.test")


(10000, 0.8942, 0.8942)

In [82]:
# get the misclassified examples by fasttext algorithm
wrong_labeled_fasttext = ""
for i in range(test_reviews.size):
    start_index = test_sentiments.index.start
    if(model.predict(test_reviews[start_index + i])[0][0] != str('__label__' + test_sentiments[start_index + i])):
        wrong_labeled_fasttext += str(start_index + i) + " " + str(imdb_data['sentiment'][start_index + i]) + " | " + str(imdb_data['review'][start_index + i]) +  "\n"

f = open("fastText Wrongly labeled sentences.txt", "w")
f.write(wrong_labeled_fasttext)
f.close


<function TextIOWrapper.close()>

# LSTM with pytorch
https://www.kaggle.com/code/fantaszzhang/deep-learning-for-sentiment-analysis/edit